In [ ]:
# You need to run this cell to get things setup
%matplotlib inline

import matplotlib
import matplotlib.pyplot as pyplot

import geopandas

import copy

# Using `geopandas` as a GIS
In this notebook we look at the ways in which `geopandas` supports standard (vector) GIS operations, such as overlay, buffering, clipping, and so on. This is as preparation for the last `geopandas` based assignment of the trimester before we dive into customizing QGIS.

Much of the material in this notebook is based closely on the tutorial documentation on the geopandas website, which you will find at [geopandas.org/index.html](http://geopandas.org/index.html).

## What we have learned so far
So far we can read geospatial files, write them, reproject them, and make simple maps with `geopandas`. That's the beginnings of a full GIS functionality, but really we need to know how to manipulate geospatial data in a wide range of ways to really say we are 'doing GIS' when we use `geopandas`. This notebook introduces the functions you need to do this, namely

+ [Methods that make a new data layer from an existing one](#Making-new-geometries-using-uh...-geometry), such as *buffering*, *centroids*, the * convex hull*, and so on;
+ [Methods that perform spatial transformations](#Transformations) on a collection of spatial objects, such as *translation*, *rotation*, *scaling* and so on;
+ Methods that perform operations between two data layers, such as [*overlay* and *intersection*](#Overlay-and-spatial-join-operations); 
+ Methods that perform [*aggregation* and *dissolve*](#Dissolving-and-aggregating-data) between layers of differing spatial types; and
+ Methods that can be used to [*merge* data tables](#Merging-data-tables-based-on-an-attribute)

## The data
To explore these methods we need so data layers. I grabbed some points, lines and areas from Open Street Map using a QGIS plugin for this.

In [ ]:
pts = geopandas.read_file("pts.geojson")
lines = geopandas.read_file("ways.geojson")
bldgs = geopandas.read_file("buildings.geojson")

And now plot them (this is also a refresher on making plots with `geopandas` and `matplotlib`).

In [ ]:
# I have no idea why this particular figure seems to need to be so large
fig = pyplot.figure(figsize=(20,20)) 
base = fig.add_subplot(131)
bldgs.plot(ax=base, facecolor='#ddbbbb')
lines.plot(ax=base, edgecolor='r')
pts.plot(ax=base, color='k', markersize=1)

For what it's worth, these are from the area around the university. The large buildings are the Kelburn Campus.

## Making new geometries using uh... geometry
The most important thing to understand about the `geopandas` geometry operations is that most of them operate at the level of the geometry objects contained in a `GeoDataFrame`'s **geometry** attribute and produce a new `GeoSeries` rather than a new `GeoDataFrame`. That is, they make a collection of geometry objects, not a whole new dataset. This has implications for using these operations to create new datasets.

### The `buffer` operation
For example, take the `buffer` operation. You can apply `buffer()` to a `GeoDataFrame` but it makes a new `GeoSeries` not a new `GeoDataFrame`

In [ ]:
pts.buffer(10).head()

You can also apply `buffer()` to a `GeoSeries`, and you'll get the same output as if you'd applied it to the `GeoDataFrame`

In [ ]:
pts.geometry.buffer(10).head()

To make a new `GeoDataFrame` which will include all the associated attributes and data, this means you have two options. One is to make a copy of the original and assign the buffered geometries to it.

In [ ]:
# Make a copy of the original dataset
pts_b10 = copy.copy(pts)
# set the geometry of the copy to the result of performing
# some operation on the original dataset
pts_b10.geometry = pts.buffer(10)

To confirm we got what we expected, we can plot these as below.

In [ ]:
fig = pyplot.figure()
base = fig.add_subplot(111)
pts_b10.plot(ax=base, facecolor='grey')
pts.plot(ax=base, color='k', markersize=1)

And at this point, if we wished, we could write `pts_b10` to a new output file, if we wanted to keep it.

The other option is to reassign the geometry of the original dataset

In [ ]:
pts.geometry = pts.buffer(10)

You have to be pretty sure this is what you want as now you have lost the original `pts` dataset!

In [ ]:
pts.geometry.head()

Note that the geometry column now contains polygons! Let's just reset `pts` back to the data from the file, so we don't forget.

In [ ]:
pts = geopandas.read_file("pts.geojson")

Either way you get all the attributes of the original dataset. If you only want some of them, you do something like this

In [ ]:
pts_b10 = pts_b10[['full_id', 'osm_type', 'geometry']]
pts_b10.head()

If you do this, you have to be careful to ensure that the `geometry` attribute is one of the ones in the list that you ask to retain in the dataset, otherwise your `GeoDataFrame` will become just a simple `DataFrame` and re-adding the geometry can be tricky.

Anyway, we can buffer lines and polygons too, when some other options may become relevant such as `cap_style` and `join_style`; see [the shapely documentation](https://shapely.readthedocs.io/en/latest/manual.html#constructive-methods) for details.

Perhaps more interesting, with polygons we can have negative buffer distances...

In [ ]:
bldgs.buffer(20).plot(alpha=0.3).set_title("Buffered by 20m")
bldgs.buffer(-1).plot(alpha=0.3).set_title("Eroded by 1m")

I have found that a negative buffer that is too large produces errors, presumably because it attempts to shrink a polygon to less than nothing!

### `centroid`, `boundary`, `convex_hull` and `envelope`
Each of these produces convenient summary objects for the elements in a dataset.  It is important to note that these **are not functions** they are attributes of the `GeoDataFrame` and so they don't require parentheses when you request them.

In [ ]:
fig = pyplot.figure(figsize=(12,12))

ax1 = fig.add_subplot(221)
ax1.set_title("centroids")
bldgs.plot(ax=ax1, facecolor='lightgrey')
bldgs.centroid.plot(ax=ax1, markersize=1, color='k')

ax2 = fig.add_subplot(222)
ax2.set_title("envelope, basically a bounding box")
bldgs.plot(ax=ax2, facecolor='grey')
bldgs.envelope.plot(ax=ax2, facecolor='r', alpha=0.3, linewidth=0)

ax3 = fig.add_subplot(223)
ax3.set_title("convex hull")
bldgs.plot(ax=ax3, facecolor='grey')
bldgs.convex_hull.plot(ax=ax3, facecolor='r', alpha=0.3, linewidth=0)

# This one looks like it does nothing, but what it does is to 
# turn a polygon (filled) into a polyline (just an outline)
# Note how there is no facecolor specifed for this
ax4 = fig.add_subplot(224)
ax4.set_title("boundary")
bldgs.plot(ax=ax4, facecolor='lightgrey')
bldgs.boundary.plot(ax=ax4, edgecolor='r')

You can also merge all the geometries in a dataset into a single geometry (`unary_union`) and to simplify the geometry of objects (`simplify()`). Try these in the cell below.

## Transformations
Because the `shapely` library on which the `geopandas` geometry operations are based is concerned with geometry on a flat 2D surface and knows nothing about projections, we can make use of a number of simple geometric transformations. It is probably most useful to think of these as editing tools.

In [ ]:
fig = pyplot.figure(figsize=(14,5))

ax1 = fig.add_subplot(131)
ax1.set_title("Scaled by 2")
bldgs.plot(ax=ax1, facecolor='lightgrey')
bldgs.scale(2, 2).plot(ax=ax1, facecolor='None', edgecolor='r')

ax2 = fig.add_subplot(132)
ax2.set_title("Rotated 45 degrees")
bldgs.plot(ax=ax2, facecolor='lightgrey')
bldgs.rotate(45).plot(ax=ax2, facecolor='None', edgecolor='r')

ax3 = fig.add_subplot(133)
ax3.set_title("Shifted 50m north and 50m east")
bldgs.plot(ax=ax3, facecolor='lightgrey')
bldgs.translate(50, 50).plot(ax=ax3, facecolor='None', edgecolor='r')

For many of these operations, if you want to apply different operations to individual elements, things are trickier, because we have to work with individual shapely geometry objects one by one using the functions in `shapely.affinity`. Here's an example, so you can see how this works.

In [ ]:
# we need the random module to generate random rotations
# and the shapely.affinity module to apply the rotate function
# to individual geometry object. So... import them
import random
import shapely.affinity

# make an empty list to put the results in
bldgs_rot = []
# now go through the individual geometries in a loop
# applying a different transformation to each one
for b in bldgs.geometry:
    bldgs_rot.append(shapely.affinity.rotate(b, random.random() * 360))

# make the resulting list into a GeoSeries
bldgs_rot = geopandas.GeoSeries(bldgs_rot)
    
# And now plot them    
fig = pyplot.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_title("Higgeldy piggeldy random rotations")
bldgs.plot(ax=ax, facecolor='lightgrey')
bldgs_rot.plot(ax=ax, facecolor='None', edgecolor='r')

If you wanted to buffer a set of objects to different distances depending for example, on building type or some other factor, then you would have to deploy an approach similar to this.

## Overlay and spatial join operations
The class GIS operation *par excellence* is overlay of various kinds. These are applied between two data layers, and they are also `GeoDataFrame` operations. In `geopandas` these are `GeoDataFrame` operations so the data tables also get involved.

### Make some datasets
Before getting into these, we really need overlapping polygon datasets, rather than the ones we've been using so far. Probably the easiest thing to do here is make some datasets. Here's a function that will make a `GeoDataFrame` with `n**2` squares of varying sizes arranged in a grid with random offsets.

In [ ]:
def make_squares(n=8, scale_range=(1/6, 2/3), rnd=2/3):
    polys = []
    min_size = scale_range[0]
    p = shapely.geometry.Polygon([(0,0), (0,min_size), (min_size,min_size), (min_size,0)])
    for dx in range(n):
        for dy in range(n):
            p2 = shapely.affinity.translate(p, dx + random.random() * rnd, dy + random.random() * rnd)
            max_scale = scale_range[1] / scale_range[0]
            s = 1 + (random.random() * (max_scale - 1))
            p2 = shapely.affinity.scale(p2, s, s)
            polys.append(p2)
    gs = geopandas.GeoSeries(polys)

    squares = geopandas.GeoDataFrame(geometry=gs)
    squares.crs = pts.crs
    return squares

Use this to make a couple of datasets.  Also give these an attribute that is clearly distinguishable between the two datasets

In [ ]:
# now make a couple of squares datasets
s1 = make_squares()
s2 = make_squares()
# also give them another attribute so we can see 
# what happens to attributes in a overlay operations
s1['A'] = range(s1.shape[0])
s2['B'] = range(s2.shape[0])
s2.B = s2.B + 1000

We'll also make a couple of matching centroid datasets.

In [ ]:
# Also make a couple of centroid datasets
c1 = copy.copy(s1)
c1.geometry = c1.centroid
c2 = copy.copy(s2)
c2.geometry = c2.centroid

Now plot the two sets of squares so we can see the relationships between them, before we do some overlays.

In [ ]:
fig = pyplot.figure(figsize=(8,8))
ax = fig.add_subplot(111)
s1.plot(ax=ax, facecolor='b', alpha=0.3)
for p, label in zip(c1.geometry, c1.A):
    ax.annotate(xy=(p.x-0.4, p.y-0.25), s=label)
s2.plot(ax=ax, facecolor='r', alpha=0.3)

### `overlay` operations
OK. So let's see what happens when we use the `geopandas.overlay` function on these datasets.

In [ ]:
s1_o_s2 = geopandas.overlay(s1, s2, how="intersection")

fig = pyplot.figure(figsize=(8,8))
ax = fig.add_subplot(111)
s1.plot(ax=ax, facecolor='None', edgecolor='b', linewidth=2)
s2.plot(ax=ax, facecolor='None', edgecolor='r', linewidth=2)
s1_o_s2.plot(ax=ax, facecolor='m', linewidth=0, alpha=0.5)
for p, label in zip(s1_o_s2.geometry, s1_o_s2.A):
    ax.annotate(xy=(p.centroid.x-0.4, p.centroid.y-0.25), s=label)

So spatially, this has produced a `GeoDataFrame` that is the intersection of the two input layers. In attribute terms, we need to see what has happened also

In [ ]:
s1_o_s2

So elements in the new dataset inherit the attributes of both the input datasets.  The `how` option sent to the `overlay` function yields different results. The options are listed [here](http://geopandas.org/reference/geopandas.overlay.html#geopandas.overlay). Give them each a try in the cells above and see what happens.  It's worth saying that not all of these will make sense in all situations.

Also worth noting that `overlay` can currently only be applied between two polygon layers.

### `sjoin` operations
Overlay operations cause changes in the geometry of the included elements. Spatial join operations which are invoked by the `geopandas.sjoin()` function usually do not, but will append attributes from one dataset on to another depending on the spatial relation between the datasets.

In [ ]:
s1_j_s2 = geopandas.sjoin(s1, s2, how='inner', op='intersects')

fig = pyplot.figure(figsize=(8,8))
ax = fig.add_subplot(111)
s1.plot(ax=ax, facecolor='None', edgecolor='b', linewidth=2)
s2.plot(ax=ax, facecolor='None', edgecolor='r', linewidth=2)
s1_j_s2.plot(ax=ax, facecolor='m', linewidth=0, alpha=0.5)
for p, label in zip(s1_j_s2.geometry, s1_j_s2.A):
    ax.annotate(xy=(p.centroid.x-0.4, p.centroid.y-0.25), s=label)

In [ ]:
s1_j_s2

Here, the options available are `how` can be `inner` (the default), `left`, or `right`. When set to `inner` only the cases in the first `GeoDataFrame` that satisfy the spatial constraint specified by the `op` setting are retained, and they acquire relevant attributes from both datasets.  When `how` is set to `left` or `right` all elements in the specified dataset are retained in the output, but there will be not data available to be joined unless the spatial constrain is met.

The spatial constraint specified by `op` can be any of `intersects`, `within` or `contains`.  Again, try experimenting with these options above to see what happens.

Spatial join operations work between different types of geometries in the expected ways.

By combining `sjoin`, `overlay` and operations such as `buffer` a very wide variety of spatial relationships between datasets can be explored. 

## Dissolving and aggregating data
There are yet more spatial operations available. An important one is to dissolve spatial elements together based on some attribute, so that all elements that share the same value of that attribute are combined into a single elements and their associated data values are combined in some manner.

Again, it is useful to make a dataset to demonstrate this.

In [ ]:
# More data
s = make_squares(8, scale_range=(1,1), rnd=0)
s['id'] = range(s.shape[0])
s['value'] = [random.randint(100,200) for i in range(s.shape[0])]
s['row'] = s.id % 8
s['col'] = s.id // 8
s.plot(column='value', cmap="Reds", edgecolor='w')

In [ ]:
d = s.dissolve(by='row', aggfunc='median', as_index=False)

In [ ]:
d

In [ ]:
d.plot(column='value', cmap='Reds', edgecolor='w')

A variety of `aggfunc` options are available: `mean`, `median`, `prod`, `sum`, `std`, `var`. Experiment with them above.

## Merging data tables based on an attribute
A final useful, albeit non-spatial capability is to merge the data tables based on shared values of a particular attribute.  This is accomplished with the `pandas` `merge` function, and an example is provided [here](http://geopandas.org/mergingdata.html#attribute-joins). More complete documentation is available [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)

## Wrapping up
This has been a whirlwind tour of pretty much all that `geopandas` has to offer. It is effectively a fully functional vector GIS (minus network analysis stuff) which can be used for many of the spatial data manipulation and management tasks that a desktop GIS such as ArcGIS or QGIS are used for.

The assignment this week asks you to use this capability to accomplish a simple GIS task.